In [2]:
import requests

In [195]:
requests.get('http://localhost:8090/').json()

{'error': 'Please specify command'}

In [232]:
texts = [
    'גנן גידל דגן בגן\nדגן גדול גדל בגן',
    'אובאמה הכריז אתמול בעצרת האו"ם שטראמפ ליצן.',
    'מלך השערים במונדיאל 2006 היה מירוסלב קלוזה.'
]

## Run NER Model

In [233]:
%%time 
payload = { 'sentences': texts[1],
            'model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/run_ner_model', params=payload).json()
list(zip(res['tokenized_text'][0], res['nemo_predictions'][0]))

CPU times: user 9.48 ms, sys: 1.08 ms, total: 10.6 ms
Wall time: 75.6 ms


[('אובאמה', 'S-PER'),
 ('הכריז', 'O'),
 ('אתמול', 'O'),
 ('בעצרת', 'O^O'),
 ('האו"ם', 'B-ORG^E-ORG'),
 ('שטראמפ', 'O^O'),
 ('ליצן', 'O'),
 ('.', 'O')]

## MD + NER

### YAP API calls

In [246]:
%%time 
payload = { 'sentences': texts[1],
            #'model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/multi_align_hybrid', params=payload).json()
res

CPU times: user 9.8 ms, sys: 2.13 ms, total: 11.9 ms
Wall time: 185 ms


{'tokenized_text': [['אובאמה',
   'הכריז',
   'אתמול',
   'בעצרת',
   'האו"ם',
   'שטראמפ',
   'ליצן',
   '.']],
 'nemo_multi_predictions': [['S-PER',
   'O',
   'O',
   'O^O',
   'B-ORG^E-ORG',
   'O^O',
   'O',
   'O']],
 'ma_lattice': '0\t1\tאובאמה\tאובאמה\tNNP\tNNP\t_\t1\n1\t2\tהכריז\tהכריז\tVB\tVB\tgen=M|num=S|per=3|tense=PAST\t2\n2\t3\tאתמול\tאתמול\tRB\tRB\t_\t3\n3\t4\tב\tב\tPREPOSITION\tPREPOSITION\t_\t4\n3\t6\tבעצרת\tבעצרת\tNNP\tNNP\tgen=M|num=S\t4\n3\t6\tבעצרת\tבעצרת\tNN\tNN\tgen=M|num=P|num=S\t4\n3\t6\tבעצרת\tבעצרת\tNN\tNN\tgen=M|num=S\t4\n3\t6\tבעצרת\tבעצרת\tNNP\tNNP\tgen=F|num=S\t4\n3\t6\tבעצרת\tבעצרת\tNNP\tNNP\tgen=F|gen=M|num=S\t4\n3\t6\tבעצרת\tבעצרת\tNNP\tNNP\t_\t4\n3\t6\tבעצרת\tבעצרת\tNN\tNN\tgen=M|num=P\t4\n3\t6\tבעצרת\tבעצרת\tNN\tNN\tgen=F|num=S\t4\n3\t6\tבעצרת\tבעצרת\tNN\tNN\tgen=F|num=P\t4\n4\t5\tה\tה\tDEF\tDEF\t_\t4\n4\t6\tעצרת\tעצר\tVB\tVB\tgen=M|num=S|per=2|tense=PAST\t4\n4\t6\tעצרת\tעצרת\tNN\tNN\tgen=F|num=S\t4\n4\t6\tעצרת\tעצרת\tNNT\tNNT\tgen=F|num=S\t4\n4\t6\t

In [236]:
import json
#data = f'{{"amblattice": "{ma["ma_lattice"]}"}}'.replace('\t','\\t').replace('\n','\\n').encode('utf-8') 
data = json.dumps({'amblattice': ma['ma_lattice']})
resp = requests.get('http://localhost:8000/yap/heb/md', data=data, headers=headers).json()
resp

{'md_lattice': '0\t1\tמלך\tמלך\tNNT\tNNT\tgen=M|num=S\t1\n1\t2\tה\tה\tDEF\tDEF\t_\t2\n2\t3\tשערים\tשער\tNN\tNN\tgen=M|num=P\t2\n3\t4\tב\tב\tPREPOSITION\tPREPOSITION\t_\t3\n4\t5\tמונדיאל\tמונדיאל\tNNT\tNNT\tgen=M|num=S\t3\n5\t6\t2006\t_\tCD\tCD\t_\t4\n6\t7\tהיה\tהיה\tCOP\tCOP\tgen=M|num=S|per=3\t5\n7\t8\tמירוסלב\tמירוסלב\tNNP\tNNP\tgen=M|num=S\t6\n8\t9\tקלוזה.\tקלוזה.\tNNP\tNNP\tgen=F|gen=M|num=S\t7\n\n'}